# Number 1

In [ ]:
class Dense_Layer:
    """
    Dense Layer class for neural networks without using any imports
    """
    
    def __init__(self):
        """Initialize the Dense Layer"""
        self.inputs = None
        self.weights = None
        self.bias = None
        self.weighted_sum = None
        self.output = None
        self.activation_type = None
    
    # a) Function to setup/accept the inputs and weights
    def setup(self, inputs, weights, bias):
        """
        Setup and accept inputs, weights, and bias
        
        Args:
            inputs: list of input values
            weights: 2D list (matrix) of weights
            bias: list of bias values
        """
        self.inputs = inputs
        self.weights = weights
        self.bias = bias
        print(f"Layer configured:")
        print(f"  Inputs: {inputs}")
        print(f"  Weights shape: {len(weights)}x{len(weights[0]) if weights else 0}")
        print(f"  Bias: {bias}")
    
    # b) Function to perform the weighted sum + bias
    def calculate_weighted_sum(self):
        """
        Calculate weighted sum: Z = W * X + b
        Performs matrix multiplication manually
        
        Returns:
            List of weighted sum values
        """
        if self.inputs is None or self.weights is None or self.bias is None:
            raise ValueError("Must call setup() first")
        
        # Number of neurons (rows in weight matrix)
        num_neurons = len(self.weights)
        
        # Initialize result
        result = []
        
        # For each neuron
        for i in range(num_neurons):
            weighted_sum = 0
            # Multiply weights by inputs
            for j in range(len(self.inputs)):
                weighted_sum += self.weights[i][j] * self.inputs[j]
            # Add bias
            weighted_sum += self.bias[i]
            result.append(weighted_sum)
        
        self.weighted_sum = result
        print(f"\nWeighted Sum (Z): {result}")
        return result
    
    # c) Function to perform the selected activation function
    def activation(self, activation_type):
        """
        Apply activation function to weighted sum
        
        Args:
            activation_type: string - 'relu', 'sigmoid', 'softmax', 'linear', 'step'
        
        Returns:
            List of activated values
        """
        if self.weighted_sum is None:
            raise ValueError("Must call calculate_weighted_sum() first")
        
        self.activation_type = activation_type
        
        if activation_type.lower() == 'relu':
            self.output = self._relu(self.weighted_sum)
        elif activation_type.lower() == 'sigmoid':
            self.output = self._sigmoid(self.weighted_sum)
        elif activation_type.lower() == 'softmax':
            self.output = self._softmax(self.weighted_sum)
        elif activation_type.lower() == 'linear':
            self.output = self._linear(self.weighted_sum)
        elif activation_type.lower() == 'step':
            self.output = self._step(self.weighted_sum)
        else:
            raise ValueError(f"Unknown activation function: {activation_type}")
        
        print(f"Activation ({activation_type}): {self.output}")
        return self.output
    
    def _relu(self, values):
        """ReLU activation: max(0, x)"""
        return [max(0, x) for x in values]
    
    def _sigmoid(self, values):
        """Sigmoid activation: 1 / (1 + e^(-x))"""
        result = []
        for x in values:
            # Calculate e^(-x) using Taylor series approximation
            exp_neg_x = self._exp(-x)
            sigmoid_value = 1 / (1 + exp_neg_x)
            result.append(sigmoid_value)
        return result
    
    def _softmax(self, values):
        """Softmax activation: e^(xi) / sum(e^(xj))"""
        # Calculate e^x for all values
        exp_values = [self._exp(x) for x in values]
        # Sum of all exponentials
        sum_exp = sum(exp_values)
        # Normalize
        return [exp_val / sum_exp for exp_val in exp_values]
    
    def _linear(self, values):
        """Linear activation: f(x) = x"""
        return values
    
    def _step(self, values, threshold=0):
        """Step activation: 1 if x >= threshold, else 0"""
        return [1 if x >= threshold else 0 for x in values]
    
    def _exp(self, x):
        """
        Calculate e^x using Taylor series expansion
        e^x = 1 + x + x^2/2! + x^3/3! + x^4/4! + ...
        """
        # For numerical stability, limit x
        if x > 20:
            x = 20
        elif x < -20:
            x = -20
        
        result = 1.0
        term = 1.0
        
        # Use 20 terms for good approximation
        for n in range(1, 20):
            term *= x / n
            result += term
        
        return result
    
    # d) Function to calculate the loss
    def calculate_loss(self, target_output, loss_type='mse'):
        """
        Calculate loss between predicted output and target output
        
        Args:
            target_output: list of target values
            loss_type: 'mse', 'mae', 'binary_crossentropy', 'categorical_crossentropy'
        
        Returns:
            Loss value
        """
        if self.output is None:
            raise ValueError("Must call activation() first")
        
        if len(self.output) != len(target_output):
            raise ValueError("Output and target must have same length")
        
        if loss_type.lower() == 'mse':
            loss = self._mse(self.output, target_output)
        elif loss_type.lower() == 'mae':
            loss = self._mae(self.output, target_output)
        elif loss_type.lower() == 'binary_crossentropy':
            loss = self._binary_crossentropy(self.output, target_output)
        elif loss_type.lower() == 'categorical_crossentropy':
            loss = self._categorical_crossentropy(self.output, target_output)
        else:
            raise ValueError(f"Unknown loss function: {loss_type}")
        
        print(f"\nLoss ({loss_type}): {loss}")
        return loss
    
    def _mse(self, predicted, target):
        """Mean Squared Error"""
        squared_errors = [(p - t) ** 2 for p, t in zip(predicted, target)]
        return sum(squared_errors) / len(squared_errors)
    
    def _mae(self, predicted, target):
        """Mean Absolute Error"""
        absolute_errors = [abs(p - t) for p, t in zip(predicted, target)]
        return sum(absolute_errors) / len(absolute_errors)
    
    def _binary_crossentropy(self, predicted, target):
        """Binary Cross Entropy Loss"""
        epsilon = 1e-7  # Small value to avoid log(0)
        loss = 0
        for p, t in zip(predicted, target):
            p = max(min(p, 1 - epsilon), epsilon)  # Clip values
            loss += -(t * self._log(p) + (1 - t) * self._log(1 - p))
        return loss / len(predicted)
    
    def _categorical_crossentropy(self, predicted, target):
        """Categorical Cross Entropy Loss"""
        epsilon = 1e-7
        loss = 0
        for p, t in zip(predicted, target):
            p = max(p, epsilon)  # Avoid log(0)
            loss += -t * self._log(p)
        return loss
    
    def _log(self, x):
        """Natural logarithm using Taylor series around x=1"""
        if x <= 0:
            return -100  # Avoid log of non-positive
        
        # For x close to 1, use ln(x) = 2 * sum((((x-1)/(x+1))^(2n+1))/(2n+1))
        # For other x, use properties: ln(x) = ln(a * b) = ln(a) + ln(b)
        
        # Reduce x to range [0.5, 1.5] for better convergence
        exponent = 0
        while x > 1.5:
            x /= 2.718281828459045  # e
            exponent += 1
        while x < 0.5:
            x *= 2.718281828459045  # e
            exponent -= 1
        
        # Taylor series for ln(x) around 1: ln(x) = (x-1) - (x-1)^2/2 + (x-1)^3/3 - ...
        y = x - 1
        result = 0
        term = y
        
        for n in range(1, 50):
            result += term / n
            term *= -y
        
        return result + exponent


# Example usage and testing
print("=" * 60)
print("DENSE LAYER CLASS - TESTING")
print("=" * 60)

# Test example with simple data
layer1 = Dense_Layer()

# Setup inputs and weights
inputs = [1.0, 2.0, 3.0]
weights = [
    [0.2, 0.8, -0.5],
    [0.5, -0.91, 0.26]
]
bias = [2.0, 3.0]

layer1.setup(inputs, weights, bias)
layer1.calculate_weighted_sum()
layer1.activation('relu')
target = [3.0, 1.0]
layer1.calculate_loss(target, 'mse')


# Number 2 a.)


In [ ]:
# Using the Dense_Layer class from Problem 1

# Initialize layers
hidden_layer_1 = Dense_Layer()
hidden_layer_2 = Dense_Layer()
output_layer = Dense_Layer()

# Input data
inputs = [5.1, 3.5, 1.4, 0.2]
target_output = [0.7, 0.2, 0.1]

print("=" * 70)
print("IRIS DATASET CLASSIFICATION - NEURAL NETWORK FORWARD PASS")
print("=" * 70)

#  FIRST HIDDEN LAYER (ReLU) 
print("\n" + "=" * 70)
print("LAYER 1 - FIRST HIDDEN LAYER")
print("=" * 70)

W1 = [
    [0.2, 0.5, -0.3, 0.1],
    [-0.2, 0.4, -0.4, 0.3],
    [0.2, 0.6, -0.1, 0.5]
]
B1 = [3.0, -2.1, 0.6]

hidden_layer_1.setup(inputs, W1, B1)
z1 = hidden_layer_1.calculate_weighted_sum()
a1 = hidden_layer_1.activation('relu')

#  SECOND HIDDEN LAYER (Sigmoid) 
print("\n" + "=" * 70)
print("LAYER 2 - SECOND HIDDEN LAYER")
print("=" * 70)

W2 = [
    [0.3, -0.5, 0.7],
    [0.2, -0.6, 0.4]
]
B2 = [4.3, 6.4]

hidden_layer_2.setup(a1, W2, B2)
z2 = hidden_layer_2.calculate_weighted_sum()
a2 = hidden_layer_2.activation('sigmoid')

#  OUTPUT LAYER (Softmax) 
print("\n" + "=" * 70)
print("LAYER 3 - OUTPUT LAYER")
print("=" * 70)

W3 = [
    [0.5, -0.3],
    [0.8, -0.2],
    [0.6, -0.4]
]
B3 = [-1.5, 2.1, -3.3]

output_layer.setup(a2, W3, B3)
z3 = output_layer.calculate_weighted_sum()
predictions = output_layer.activation('softmax')

#  CALCULATE LOSS 
print("\n" + "=" * 70)
print("LOSS CALCULATION")
print("=" * 70)
print(f"\nPredicted Output: {predictions}")
print(f"Target Output: {target_output}")

loss = output_layer.calculate_loss(target_output, 'categorical_crossentropy')

#  FINAL RESULTS 
print("\n" + "=" * 70)
print("CLASSIFICATION RESULTS")
print("=" * 70)

species = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
print(f"\n{'Class':<20} {'Probability':<15} {'Confidence %':<15}")
print("-" * 50)

max_prob = max(predictions)
max_index = predictions.index(max_prob)

for i, prob in enumerate(predictions):
    print(f"{species[i]:<20} {prob:<15.6f} {prob*100:<15.2f}%")

print("-" * 50)
print(f"\nPredicted Class: {species[max_index]}")
print(f"Confidence: {max_prob*100:.2f}%")
print(f"Categorical Cross Entropy Loss: {loss:.6f}")

print("\n" + "=" * 70)


# Number 2 b.)

In [25]:
class Dense_Layer:
    """Simplified Dense Layer for forward pass"""
    
    def __init__(self):
        self.inputs = None
        self.weights = None
        self.bias = None
        self.weighted_sum = None
        self.output = None
    
    def setup(self, inputs, weights, bias):
        """Setup inputs, weights, and bias"""
        self.inputs = inputs
        self.weights = weights
        self.bias = bias
    
    def calculate_weighted_sum(self):
        """Calculate Z = W * X + b"""
        num_neurons = len(self.weights)
        result = []
        
        for i in range(num_neurons):
            weighted_sum = 0
            for j in range(len(self.inputs)):
                weighted_sum += self.weights[i][j] * self.inputs[j]
            weighted_sum += self.bias[i]
            result.append(weighted_sum)
        
        self.weighted_sum = result
        return result
    
    def activation(self, activation_type):
        """Apply activation function"""
        if activation_type.lower() == 'relu':
            self.output = [max(0, x) for x in self.weighted_sum]
        elif activation_type.lower() == 'sigmoid':
            self.output = [self._sigmoid(x) for x in self.weighted_sum]
        return self.output
    
    def _sigmoid(self, x):
        """Sigmoid: 1 / (1 + e^(-x))"""
        if x > 20:
            x = 20
        elif x < -20:
            x = -20
        exp_neg_x = self._exp(-x)
        return 1 / (1 + exp_neg_x)
    
    def _exp(self, x):
        """Calculate e^x using Taylor series"""
        if x > 20:
            x = 20
        elif x < -20:
            x = -20
        
        result = 1.0
        term = 1.0
        
        for n in range(1, 20):
            term *= x / n
            result += term
        
        return result
    
    def calculate_loss(self, target_output, loss_type='binary_crossentropy'):
        """Calculate loss"""
        if loss_type.lower() == 'binary_crossentropy':
            loss = 0
            epsilon = 1e-7
            for p, t in zip(self.output, target_output):
                p = max(min(p, 1 - epsilon), epsilon)
                loss += -(t * self._log(p) + (1 - t) * self._log(1 - p))
            return loss / len(self.output)
    
    def _log(self, x):
        """Natural logarithm approximation"""
        if x <= 0:
            return -100
        
        exponent = 0
        while x > 1.5:
            x /= 2.718281828459045
            exponent += 1
        while x < 0.5:
            x *= 2.718281828459045
            exponent -= 1
        
        y = x - 1
        result = 0
        term = y
        
        for n in range(1, 50):
            result += term / n
            term *= -y
        
        return result + exponent


# BREAST CANCER CLASSIFICATION - NEURAL NETWORK


print("=" * 75)
print("BREAST CANCER DATASET CLASSIFICATION - NEURAL NETWORK FORWARD PASS")
print("=" * 75)

# Initialize layers
hidden_layer_1 = Dense_Layer()
hidden_layer_2 = Dense_Layer()
output_layer = Dense_Layer()

# Input data
inputs = [14.1, 20.3, 0.095]
target_output = [1]

print("\nInput Features:")
print(f"  Mean Radius: {inputs[0]}")
print(f"  Mean Texture: {inputs[1]}")
print(f"  Mean Smoothness: {inputs[2]}")
print(f"Target Output: {target_output[0]} (Malignant)")

#  FIRST HIDDEN LAYER (ReLU) 
print("\n" + "=" * 75)
print("LAYER 1 - FIRST HIDDEN LAYER (ReLU Activation)")
print("=" * 75)

W1 = [
    [0.5, -0.3, 0.8],
    [0.2, 0.4, -0.6],
    [-0.7, 0.9, 0.1]
]
B1 = [0.3, -0.5, 0.6]

print("\nWeights W1:")
for i, row in enumerate(W1):
    print(f"  Node {i+1}: {row}")
print(f"Bias B1: {B1}")

hidden_layer_1.setup(inputs, W1, B1)
z1 = hidden_layer_1.calculate_weighted_sum()

print(f"\nWeighted Sum (Z1):")
for i, val in enumerate(z1):
    calculation = f"({W1[i][0]}×{inputs[0]}) + ({W1[i][1]}×{inputs[1]}) + ({W1[i][2]}×{inputs[2]}) + {B1[i]}"
    print(f"  Node {i+1}: {val:.4f}")
    print(f"    Calculation: {calculation}")

a1 = hidden_layer_1.activation('relu')
print(f"\nAfter ReLU Activation (A1): {a1}")

#  SECOND HIDDEN LAYER (Sigmoid) 
print("\n" + "=" * 75)
print("LAYER 2 - SECOND HIDDEN LAYER (Sigmoid Activation)")
print("=" * 75)

W2 = [
    [0.6, -0.2, 0.4],
    [-0.3, 0.5, 0.7]
]
B2 = [0.1, -0.8]

print("\nWeights W2:")
for i, row in enumerate(W2):
    print(f"  Node {i+1}: {row}")
print(f"Bias B2: {B2}")

hidden_layer_2.setup(a1, W2, B2)
z2 = hidden_layer_2.calculate_weighted_sum()

print(f"\nWeighted Sum (Z2):")
for i, val in enumerate(z2):
    calculation = f"({W2[i][0]}×{a1[0]:.4f}) + ({W2[i][1]}×{a1[1]:.4f}) + ({W2[i][2]}×{a1[2]:.4f}) + {B2[i]}"
    print(f"  Node {i+1}: {val:.4f}")
    print(f"    Calculation: {calculation}")

a2 = hidden_layer_2.activation('sigmoid')
print(f"\nAfter Sigmoid Activation (A2):")
for i, val in enumerate(a2):
    print(f"  Node {i+1}: {val:.6f}")

#  OUTPUT LAYER (Sigmoid) 
print("\n" + "=" * 75)
print("LAYER 3 - OUTPUT LAYER (Sigmoid Activation)")
print("=" * 75)

W3 = [[0.7, -0.5]]
B3 = [0.2]

print("\nWeights W3:")
for i, row in enumerate(W3):
    print(f"  Output Node: {row}")
print(f"Bias B3: {B3}")

output_layer.setup(a2, W3, B3)
z3 = output_layer.calculate_weighted_sum()

print(f"\nWeighted Sum (Z3):")
calculation = f"({W3[0][0]}×{a2[0]:.6f}) + ({W3[0][1]}×{a2[1]:.6f}) + {B3[0]}"
print(f"  Output Node: {z3[0]:.4f}")
print(f"    Calculation: {calculation}")

predictions = output_layer.activation('sigmoid')
print(f"\nAfter Sigmoid Activation (Predictions): {predictions[0]:.6f}")

#  CALCULATE LOSS 
print("\n" + "=" * 75)
print("LOSS CALCULATION")
print("=" * 75)

loss = output_layer.calculate_loss(target_output, 'binary_crossentropy')

print(f"\nPredicted Output: {predictions[0]:.6f}")
print(f"Target Output: {target_output[0]}")
print(f"Binary Cross Entropy Loss: {loss:.6f}")

#  FINAL CLASSIFICATION RESULT 
print("\n" + "=" * 75)
print("CLASSIFICATION RESULT")
print("=" * 75)

probability_malignant = predictions[0]
probability_benign = 1 - probability_malignant
threshold = 0.5

print(f"\nProbability Malignant (1): {probability_malignant:.6f} ({probability_malignant*100:.2f}%)")
print(f"Probability Benign (0): {probability_benign:.6f} ({probability_benign*100:.2f}%)")
print(f"Decision Threshold: {threshold}")

if probability_malignant >= threshold:
    prediction_class = "MALIGNANT (1)"
else:
    prediction_class = "BENIGN (0)"

print(f"\nPredicted Classification: {prediction_class}")
print(f"Confidence: {max(probability_malignant, probability_benign)*100:.2f}%")

print("\n" + "=" * 75)


BREAST CANCER DATASET CLASSIFICATION - NEURAL NETWORK FORWARD PASS

Input Features:
  Mean Radius: 14.1
  Mean Texture: 20.3
  Mean Smoothness: 0.095
Target Output: 1 (Malignant)

LAYER 1 - FIRST HIDDEN LAYER (ReLU Activation)

Weights W1:
  Node 1: [0.5, -0.3, 0.8]
  Node 2: [0.2, 0.4, -0.6]
  Node 3: [-0.7, 0.9, 0.1]
Bias B1: [0.3, -0.5, 0.6]

Weighted Sum (Z1):
  Node 1: 1.3360
    Calculation: (0.5×14.1) + (-0.3×20.3) + (0.8×0.095) + 0.3
  Node 2: 10.3830
    Calculation: (0.2×14.1) + (0.4×20.3) + (-0.6×0.095) + -0.5
  Node 3: 9.0095
    Calculation: (-0.7×14.1) + (0.9×20.3) + (0.1×0.095) + 0.6

After ReLU Activation (A1): [1.336, 10.383000000000001, 9.0095]

LAYER 2 - SECOND HIDDEN LAYER (Sigmoid Activation)

Weights W2:
  Node 1: [0.6, -0.2, 0.4]
  Node 2: [-0.3, 0.5, 0.7]
Bias B2: [0.1, -0.8]

Weighted Sum (Z2):
  Node 1: 2.4288
    Calculation: (0.6×1.3360) + (-0.2×10.3830) + (0.4×9.0095) + 0.1
  Node 2: 10.2973
    Calculation: (-0.3×1.3360) + (0.5×10.3830) + (0.7×9.0095) + -0